In [1]:
""" This is the main script for eap ml project, which plays with all ml methods, applicable for cpcrsp analysis
Created on Sat Feb 12 15:19:54 2022
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import random
import gc
import dill
import statsmodels.api as sm
from sklearn.svm import SVR
from tensorflow.keras.callbacks import EarlyStopping


from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNetCV
from sklearn import svm
from sklearn.decomposition import PCA, KernelPCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

plt.style.use('seaborn-white')
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', 40)
gc.enable()

In [ ]:
path = '../input/mleap-data/'

s_train_X = pd.read_csv(path + 's_train_X.csv')
s_train_y = pd.read_csv(path + 's_train_y.csv')
s_test_X = pd.read_csv(path + 's_test_X.csv')
s_test_y = pd.read_csv(path + 's_test_y.csv')
sp_train_X = pd.read_csv(path + 'sp_train_X.csv')
lp_train_X = pd.read_csv(path + 'lp_train_X.csv')
sp_test_X = pd.read_csv(path + 'sp_test_X.csv')
lp_test_X = pd.read_csv(path + 'lp_test_X.csv')

s_train_X.drop(columns='Unnamed: 0', inplace=True)
s_train_y.drop(columns='Unnamed: 0', inplace=True)
s_test_X.drop(columns='Unnamed: 0', inplace=True)
sp_train_X.drop(columns='Unnamed: 0', inplace=True)
lp_train_X.drop(columns='Unnamed: 0', inplace=True)
lp_test_X.drop(columns='Unnamed: 0', inplace=True)

s_train_X = np.array(s_train_X)
s_train_y = np.array(s_train_y)
s_test_X = np.array(s_test_X)
s_test_y = np.array(s_test_y)
sp_train_X = np.array(sp_train_X)
lp_train_X = np.array(lp_train_X)
sp_test_X = np.array(sp_test_X)
lp_test_X = np.array(lp_test_X)

print(s_train_X.shape)
print(s_train_y.shape)
print(s_test_X.shape)
print(s_test_y.shape)
print(sp_train_X.shape)
print(lp_train_X.shape)
print(sp_test_X.shape)
print(lp_test_X.shape)

In [ ]:
model_ann5_s = Sequential([
    BatchNormalization(input_shape=(27,)),
    Dense(256, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.4),
    Dense(1)])


print(model_ann5_s.count_params())

In [ ]:
early_stopping20 = EarlyStopping(patience=20)
early_stopping15 = EarlyStopping(patience=15)

model_ann5_s.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
history = model_ann5_s.fit(s_train_X, s_train_y, validation_data=(s_test_X, s_test_y), 
                         batch_size=512, epochs=80, verbose=2, callbacks=[early_stopping20])
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

ann_s = [r2_score(s_train_y, model_ann5_s.predict(s_train_X)), 
       r2_score(s_test_y, model_ann5_s.predict(s_test_X))]


In [ ]:
ann_s 

In [ ]:
import pickle

with open("ann_results_2", "wb") as fp:   #Pickling
    pickle.dump(ann_results, fp)

In [ ]:
temp = pd.DataFrame(ann_results)
temp.to_csv('ann_results_2.csv')

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [ ]:
create_download_link(temp)
